# List of Exposures  at USDF


- work with Weakly_2023_44
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2023/09/13
- last update : 2023/12/20



In [1]:
import sys
print(sys.executable)

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-7.0.1/bin/python3


In [2]:
print(sys.path)

['/sdf/data/rubin/user/dagoret/AuxTelComm/notebooks_usdf/butlertools', '/home/d/dagoret/repos/repos_w_2023_44/analysis_tools/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/cp_pipe/g9dd2c6f9d4+29174bd147/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/shapelet/g44018dc512+2565ec44ba/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/meas_modelfit/gd162630629+22efd3ad40/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/scarlet_lite/g13f254ea0d+81bc2a20b4/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/proxmin/g33b4157f25/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/scarlet/g63ff1f842d+4083830bf8/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/meas_extensions_scarlet/gabf74fab76+76066ce010/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/meas_extensions_p

In [3]:
! eups list -s | grep LOCAL

analysis_tools        LOCAL:/home/d/dagoret/repos/repos_w_2023_44/analysis_tools 	setup
atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2023_44/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-7.0.1/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2023_44/Spectractor 	setup


In [4]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

Weekly 2023_44
   g4213664e8e+d053631def 	current w_2023_44 setup


In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits
from astropy.time import Time

In [6]:
from IPython.display import display, HTML

In [7]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [8]:
import lsst.daf.butler as dafButler

In [9]:
repo = "/sdf/group/rubin/repo/main"
#repo="/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [10]:
collection='LATISS/raw/all'

## List of Exposures

In [11]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num','time_start','time_end' ,'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle','azimuth','zenith','science_program','jd','mjd'])

In [12]:
print(butler.registry.dimensions["exposure"].RecordClass.fields)

exposure: 
  instrument: str
  id: int
  physical_filter: str
  obs_id: str
  exposure_time: float
  dark_time: float
  observation_type: str
  observation_reason: str
  day_obs: int
  seq_num: int
  seq_start: int
  seq_end: int
  group_name: str
  group_id: int
  target_name: str
  science_program: str
  tracking_ra: float
  tracking_dec: float
  sky_angle: float
  azimuth: float
  zenith_angle: float
  has_simulated: bool
  timespan: lsst.daf.butler.Timespan


In [13]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= "instrument='LATISS'")):
    
    
    try:
        df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,pd.to_datetime(info.timespan.begin.to_string()),pd.to_datetime(info.timespan.end.to_string()) ,info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle,info.azimuth ,info.zenith_angle, info.science_program,
                             info.timespan.begin.jd,info.timespan.begin.mjd ]
    except:
        print(">>>   Unexpected error:", sys.exc_info()[0])
        info_timespan_begin_to_string = "2021-01-01 00:00:00.00"
        info_timespan_end_to_string = "2051-01-01 00:00:00.00"
        info_timespan_begin_jd = 0
        info_timespan_begin_mjd = 0
        df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,
                                  pd.to_datetime(info_timespan_begin_to_string),
                                  pd.to_datetime(info_timespan_end_to_string) ,
                                  info.observation_type, info.target_name, 
                                  info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle,info.azimuth ,info.zenith_angle, info.science_program,
                             info_timespan_begin_jd, info_timespan_begin_mjd  ]
 
    
    if count < 5:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
        
        mjd = Time(info.timespan.begin.to_string()).mjd
        jd = Time(info.timespan.begin.to_string()).jd
        print(mjd,jd)
    

----------------------------------------------------- 0 ---------------------------------------------------------
exposure:
  instrument: 'LATISS'
  id: 2022062800004
  physical_filter: 'SDSSr~empty'
  obs_id: 'AT_O_20220628_000004'
  exposure_time: 0.0
  dark_time: 0.103406
  observation_type: 'bias'
  observation_reason: 'bias'
  day_obs: 20220628
  seq_num: 4
  seq_start: 4
  seq_end: 4
  group_name: '2022-06-28T17:03:59.508'
  group_id: 2363654395080000
  target_name: 'Park position'
  science_program: 'unknown'
  tracking_ra: None
  tracking_dec: None
  sky_angle: None
  azimuth: None
  zenith_angle: None
  has_simulated: False
  timespan: Timespan(begin=astropy.time.Time('2022-06-28 17:03:59.681996', scale='tai', format='iso'), end=astropy.time.Time('2022-06-28 17:03:59.693000', scale='tai', format='iso'))
	 id:                   2022062800004
	 day_obs:              20220628
	 seq_num:              4
	 type-of-observation:  bias
	 target:               Park position
59758.711107

In [14]:
info

exposure.RecordClass(instrument='LATISS', id=2022060900939, physical_filter='SDSSg~empty', obs_id='AT_O_20220609_000939', exposure_time=5.0, dark_time=5.32103, observation_type='acq', observation_reason='final_infocus', day_obs=20220609, seq_num=939, seq_start=939, seq_end=939, group_name='2022-06-10T10:20:00.477', group_id=2347860004770000, target_name='HD 221054', science_program='CWFS', tracking_ra=352.28536459305946, tracking_dec=-51.01959048738477, sky_angle=269.4536978266297, azimuth=None, zenith_angle=21.491897644213168, has_simulated=False, timespan=Timespan(begin=astropy.time.Time('2022-06-10 10:26:02.732005', scale='tai', format='iso'), end=astropy.time.Time('2022-06-10 10:26:07.965000', scale='tai', format='iso')))

In [15]:
df_exposure

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
0,2022062800004,AT_O_20220628_000004,20220628,4,2022-06-28 17:03:59.681996,2022-06-28 17:03:59.693,bias,Park position,SDSSr~empty,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459759e+06,59758.711107
1,2022070600001,AT_O_20220706_000001,20220706,1,2022-07-07 00:07:28.587991,2022-07-07 00:07:33.832,test,UNKNOWN,unknown~unknown,NaN,5.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459768e+06,59767.005192
2,2022070700001,AT_O_20220707_000001,20220707,1,2022-07-07 13:58:52.282014,2022-07-07 13:58:52.293,bias,UNKNOWN,empty~ronchi170lpmm,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459768e+06,59767.582550
3,2022071100001,AT_O_20220711_000001,20220711,1,2022-07-11 18:15:33.602017,2022-07-11 18:15:33.613,bias,UNKNOWN,FELH0600~holo4_003,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459772e+06,59771.760806
4,2022071200001,AT_O_20220712_000001,20220712,1,2022-07-12 15:42:44.195989,2022-07-12 15:42:44.205,bias,UNKNOWN,SDSSg~holo4_003,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459773e+06,59772.654678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44230,2022060900935,AT_O_20220609_000935,20220609,935,2022-06-10 10:22:13.077005,2022-06-10 10:22:43.316,cwfs,HD 221054,SDSSg~empty,21.707667,30.0,352.285362,-51.019608,269.453648,None,21.707667,CWFS,2.459741e+06,59740.432096
44231,2022060900936,AT_O_20220609_000936,20220609,936,2022-06-10 10:22:51.409003,2022-06-10 10:23:21.645,cwfs,HD 221054,SDSSg~empty,21.670582,30.0,352.28535,-51.019575,269.453634,None,21.670582,CWFS,2.459741e+06,59740.432539
44232,2022060900937,AT_O_20220609_000937,20220609,937,2022-06-10 10:24:10.998996,2022-06-10 10:24:41.230,cwfs,HD 221054,SDSSg~empty,21.594164,30.0,352.285373,-51.019592,269.453635,None,21.594164,CWFS,2.459741e+06,59740.433461
44233,2022060900938,AT_O_20220609_000938,20220609,938,2022-06-10 10:24:48.485013,2022-06-10 10:25:18.720,cwfs,HD 221054,SDSSg~empty,21.559853,30.0,352.285369,-51.019605,269.453759,None,21.559853,CWFS,2.459741e+06,59740.433895


In [16]:
df_exposure = df_exposure.astype({"id": int,'day_obs': int,'seq_num':int})

In [17]:
df_exposure

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
0,2022062800004,AT_O_20220628_000004,20220628,4,2022-06-28 17:03:59.681996,2022-06-28 17:03:59.693,bias,Park position,SDSSr~empty,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459759e+06,59758.711107
1,2022070600001,AT_O_20220706_000001,20220706,1,2022-07-07 00:07:28.587991,2022-07-07 00:07:33.832,test,UNKNOWN,unknown~unknown,NaN,5.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459768e+06,59767.005192
2,2022070700001,AT_O_20220707_000001,20220707,1,2022-07-07 13:58:52.282014,2022-07-07 13:58:52.293,bias,UNKNOWN,empty~ronchi170lpmm,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459768e+06,59767.582550
3,2022071100001,AT_O_20220711_000001,20220711,1,2022-07-11 18:15:33.602017,2022-07-11 18:15:33.613,bias,UNKNOWN,FELH0600~holo4_003,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459772e+06,59771.760806
4,2022071200001,AT_O_20220712_000001,20220712,1,2022-07-12 15:42:44.195989,2022-07-12 15:42:44.205,bias,UNKNOWN,SDSSg~holo4_003,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459773e+06,59772.654678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44230,2022060900935,AT_O_20220609_000935,20220609,935,2022-06-10 10:22:13.077005,2022-06-10 10:22:43.316,cwfs,HD 221054,SDSSg~empty,21.707667,30.0,352.285362,-51.019608,269.453648,None,21.707667,CWFS,2.459741e+06,59740.432096
44231,2022060900936,AT_O_20220609_000936,20220609,936,2022-06-10 10:22:51.409003,2022-06-10 10:23:21.645,cwfs,HD 221054,SDSSg~empty,21.670582,30.0,352.28535,-51.019575,269.453634,None,21.670582,CWFS,2.459741e+06,59740.432539
44232,2022060900937,AT_O_20220609_000937,20220609,937,2022-06-10 10:24:10.998996,2022-06-10 10:24:41.230,cwfs,HD 221054,SDSSg~empty,21.594164,30.0,352.285373,-51.019592,269.453635,None,21.594164,CWFS,2.459741e+06,59740.433461
44233,2022060900938,AT_O_20220609_000938,20220609,938,2022-06-10 10:24:48.485013,2022-06-10 10:25:18.720,cwfs,HD 221054,SDSSg~empty,21.559853,30.0,352.285369,-51.019605,269.453759,None,21.559853,CWFS,2.459741e+06,59740.433895


## Selection of science

In [18]:
df_science = df_exposure[df_exposure.type == 'science']

In [19]:
df_science.sort_values(by="id",ascending=True,inplace=True)

/tmp/ipykernel_7046/3986782663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science.sort_values(by="id",ascending=True,inplace=True)


In [20]:
df_science.head(60)

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
12401,2020012700453,AT_O_20200127_000453,20200127,453,2020-01-28 00:56:30.621656,2020-01-28 00:56:32.621656,science,NOTSET,KPNO_373A_677nm~empty,28.107416,2.0,83.797012,-5.457447,118.534586,None,28.107416,unknown,2.458877e+06,58876.039243
12208,2020012700454,AT_O_20200127_000454,20200127,454,2020-01-28 00:58:10.978479,2020-01-28 00:58:11.978479,science,NOTSET,KPNO_373A_677nm~empty,27.922027,1.0,83.795099,-5.458437,118.534586,None,27.922027,unknown,2.458877e+06,58876.040405
12275,2020012700455,AT_O_20200127_000455,20200127,455,2020-01-28 01:00:21.385446,2020-01-28 01:00:22.385446,science,NOTSET,KPNO_373A_677nm~empty,27.688239,1.0,83.793716,-5.459686,118.534586,None,27.688239,unknown,2.458877e+06,58876.041914
12449,2020012700456,AT_O_20200127_000456,20200127,456,2020-01-28 01:06:58.395416,2020-01-28 01:07:00.395416,science,NOTSET,KPNO_373A_677nm~empty,27.028161,2.0,83.799803,-5.463436,118.534586,None,27.028161,unknown,2.458877e+06,58876.046509
12345,2020012700457,AT_O_20200127_000457,20200127,457,2020-01-28 01:07:56.651393,2020-01-28 01:07:58.651393,science,NOTSET,KPNO_373A_677nm~empty,26.937795,2.0,83.801411,-5.463949,118.534586,None,26.937795,unknown,2.458877e+06,58876.047183
12186,2020012700458,AT_O_20200127_000458,20200127,458,2020-01-28 01:16:18.521613,2020-01-28 01:16:20.521613,science,NOTSET,KPNO_373A_677nm~empty,26.225092,2.0,83.805403,-5.46856,118.534586,None,26.225092,unknown,2.458877e+06,58876.052992
12179,2020012700459,AT_O_20200127_000459,20200127,459,2020-01-28 01:17:37.584758,2020-01-28 01:17:47.584758,science,NOTSET,KPNO_373A_677nm~empty,26.134681,10.0,83.839789,-5.469201,118.534586,None,26.134681,unknown,2.458877e+06,58876.053907
12315,2020012700460,AT_O_20200127_000460,20200127,460,2020-01-28 01:20:20.922274,2020-01-28 01:20:30.922274,science,NOTSET,KPNO_373A_677nm~empty,25.93744,10.0,83.84284,-5.47063,118.534586,None,25.93744,unknown,2.458877e+06,58876.055798
12273,2020012700461,AT_O_20200127_000461,20200127,461,2020-01-28 01:23:38.989816,2020-01-28 01:23:48.989816,science,NOTSET,KPNO_373A_677nm~empty,25.7172,10.0,83.845038,-5.472374,118.534586,None,25.7172,unknown,2.458877e+06,58876.058090
12160,2020012700462,AT_O_20200127_000462,20200127,462,2020-01-28 01:24:48.204061,2020-01-28 01:24:58.204061,science,NOTSET,KPNO_373A_677nm~empty,25.645567,10.0,83.846579,-5.472948,118.534586,None,25.645567,unknown,2.458877e+06,58876.058891


In [21]:
df_science.tail(60)

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
2065,2022063000741,AT_O_20220630_000741,20220630,741,2022-07-01 08:37:43.900991,2022-07-01 08:38:14.127,science,AUXTEL_DRP_IMAGING:SDSS-J203722_005,SDSSr~empty,36.216657,30.0,309.341775,-5.594321,50.470185,307.232368,36.216657,AUXTEL_DRP,2.459762e+06,59761.359536
2066,2022063000742,AT_O_20220630_000742,20220630,742,2022-07-01 08:38:16.453983,2022-07-01 08:38:46.689,science,AUXTEL_DRP_IMAGING:SDSS-J203722_005,SDSSr~empty,36.311557,30.0,309.341675,-5.594353,50.472142,307.064757,36.311557,AUXTEL_DRP,2.459762e+06,59761.359913
2067,2022063000743,AT_O_20220630_000743,20220630,743,2022-07-01 08:38:58.550998,2022-07-01 08:39:28.782,science,AUXTEL_DRP_IMAGING:SDSS-J203722_006,SDSSr~empty,36.36748,30.0,309.436196,-5.594359,50.230394,306.966977,36.36748,AUXTEL_DRP,2.459762e+06,59761.360400
2068,2022063000744,AT_O_20220630_000744,20220630,744,2022-07-01 08:39:31.109985,2022-07-01 08:40:01.340,science,AUXTEL_DRP_IMAGING:SDSS-J203722_006,SDSSr~empty,36.459853,30.0,309.436102,-5.594346,50.231639,306.805591,36.459853,AUXTEL_DRP,2.459762e+06,59761.360777
2069,2022063000745,AT_O_20220630_000745,20220630,745,2022-07-01 08:40:43.954007,2022-07-01 08:41:14.179,science,AUXTEL_DRP_IMAGING:A2390_016,SDSSr~empty,48.571891,30.0,328.698147,17.412737,83.126668,347.627685,48.571891,AUXTEL_DRP,2.459762e+06,59761.361620
2070,2022063000746,AT_O_20220630_000746,20220630,746,2022-07-01 08:41:16.506999,2022-07-01 08:41:46.750,science,AUXTEL_DRP_IMAGING:A2390_016,SDSSr~empty,48.597445,30.0,328.698026,17.412692,83.124157,347.455713,48.597445,AUXTEL_DRP,2.459762e+06,59761.361997
2071,2022063000747,AT_O_20220630_000747,20220630,747,2022-07-01 08:41:58.247992,2022-07-01 08:42:28.472,science,AUXTEL_DRP_IMAGING:A2390_022,SDSSr~empty,48.759167,30.0,328.50127,17.506591,82.501275,347.022909,48.759167,AUXTEL_DRP,2.459762e+06,59761.362480
2072,2022063000748,AT_O_20220630_000748,20220630,748,2022-07-01 08:42:30.799012,2022-07-01 08:43:01.034,science,AUXTEL_DRP_IMAGING:A2390_022,SDSSr~empty,48.786121,30.0,328.501103,17.506714,82.502325,346.851835,48.786121,AUXTEL_DRP,2.459762e+06,59761.362856
2073,2022063000749,AT_O_20220630_000749,20220630,749,2022-07-01 08:43:13.054989,2022-07-01 08:43:43.282,science,AUXTEL_DRP_IMAGING:A2390_023,SDSSr~empty,48.801233,30.0,328.599795,17.506644,82.299976,346.756835,48.801233,AUXTEL_DRP,2.459762e+06,59761.363346
2074,2022063000750,AT_O_20220630_000750,20220630,750,2022-07-01 08:43:45.607981,2022-07-01 08:44:15.843,science,AUXTEL_DRP_IMAGING:A2390_023,SDSSr~empty,48.82869,30.0,328.599615,17.506728,82.298997,346.586003,48.82869,AUXTEL_DRP,2.459762e+06,59761.363722


In [22]:
df_science.day_obs.unique()

array([20200127, 20200128, 20200129, 20200212, 20200217, 20200218,
       20200219, 20200220, 20200221, 20200312, 20200313, 20200314,
       20200315, 20200316, 20210119, 20210120, 20210121, 20210126,
       20210209, 20210216, 20210217, 20210218, 20210309, 20210310,
       20210311, 20210323, 20210524, 20210525, 20210608, 20210609,
       20210610, 20210706, 20210707, 20210708, 20210727, 20210804,
       20210817, 20210907, 20210908, 20210909, 20211005, 20211006,
       20211102, 20211103, 20211104, 20211130, 20220208, 20220209,
       20220215, 20220216, 20220217, 20220308, 20220311, 20220316,
       20220317, 20220405, 20220406, 20220407, 20220503, 20220504,
       20220505, 20220524, 20220525, 20220526, 20220607, 20220608,
       20220609, 20220628, 20220629, 20220630])

## Selection of date

2021-02-16  2021-02-18  2021-06-08  2021-06-10  2021-07-07  2021-09-07  2021-09-09  2021-10-06  2021-11-02  2021-11-04  2022-02-16  2022-03-15  2022-03-17  20220502  
2021-02-17  2021-03-11  2021-06-09  2021-07-06  2021-07-08  2021-09-08  2021-10-05  2021-10-07  2021-11-03  2022-02-15  2022-02-17  2022-03-16  2022-03-18  20220503 20220524 20220607 20220608 20220609

### Add those

20220504 - Not existing

In [23]:
#date_sel = 20220630
#date_sel = 20220630
#date_sel = 20221208
#date_sel = 20221209

# data in january
#date_sel= 20230118
#date_sel= 20230131

# data in February
#date_sel = 20230201
#date_sel = 20230202
#date_sel = 20230704

#date_sel = 20230510
#date_sel = 20230509
#date_sel = 20230511

#date_sel = 20230705
#date_sel = 20230706

#july 2023
#date_sel = 20230718
#date_sel = 20230720

#august 2023
#date_sel = 20230801
#date_sel = 20230802

#date_sel = 20230815
#date_sel = 20230817


#september 2023
#date_sel = 20230912
#date_sel = 20230913
#date_sel = 20230914

# October 2021
date_sel = 20211006

# November 2021
date_sel = 20211103

## Query Dimension records on Exposures

In [24]:
where_clause = f"instrument=\'LATISS\' AND exposure.day_obs={date_sel} AND exposure.science_program=\'spec\'"

In [25]:
results_exposures = registry.queryDimensionRecords('exposure',datasets="raw",collections = 'LATISS/raw/all',where=where_clause)

## Select date

In [26]:
df_science_selected = df_science[df_science.day_obs == date_sel ]

In [27]:
len(df_science_selected)

263

# List of filters

In [28]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

array(['FELH0600~empty', 'SDSSg~empty', 'FELH0600~holo4_003',
       'empty~holo4_003', 'BG40~holo4_003', 'SDSSg~holo4_003'],
      dtype=object)

In [29]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

array(['FELH0600~holo4_003', 'empty~holo4_003', 'BG40~holo4_003',
       'SDSSg~holo4_003'], dtype='<U18')

## Selection of filter

- But now we have a loop on filters

In [30]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [31]:
pd.set_option('display.max_rows', None)

In [32]:
all_def_science_selected_final = []

for selected_filter in selected_filters:

    print(selected_filter)
    df_science_selected_final = df_science_selected[df_science_selected["filter"] ==  selected_filter]
    
    #print(df_science_selected_final)
    all_def_science_selected_final.append(df_science_selected_final.copy())
    
    df = df_science_selected_final
    df_sel = df[['day_obs', 'seq_num']]
    
    
    NN = len(df)
    
    if NN>0:
        # build the line
        
        df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
    
    
         # prepare output
        filename_out = "visitdispersers_"+ str(date_sel) + "_filt_" + selected_filter.replace('~','-') + ".list"
        print(filename_out)
        
        df_out = df_sel[["line"]]
        df_out.to_csv(filename_out, index=False, header=None)

FELH0600~holo4_003
visitdispersers_20211103_filt_FELH0600-holo4_003.list
empty~holo4_003
visitdispersers_20211103_filt_empty-holo4_003.list
BG40~holo4_003
visitdispersers_20211103_filt_BG40-holo4_003.list
SDSSg~holo4_003
visitdispersers_20211103_filt_SDSSg-holo4_003.list


/tmp/ipykernel_7046/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
/tmp/ipykernel_7046/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
/tmp/ipykernel_7046/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [33]:
N = len(all_def_science_selected_final)

In [34]:
if N>=1:
    n0=len(all_def_science_selected_final[0])
    display(all_def_science_selected_final[0])

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
25236,2021110300448,AT_O_20211103_000448,20211103,448,2021-11-04 05:07:43.369530,2021-11-04 05:07:45.618,science,HD 37795,FELH0600~holo4_003,30.04624,2.0,84.912487,-34.074126,357.466664,None,30.04624,unknown,2.459523e+06,59522.213696
25237,2021110300449,AT_O_20211103_000449,20211103,449,2021-11-04 05:08:20.216512,2021-11-04 05:08:22.448,science,HD 37795,FELH0600~holo4_003,29.922213,2.0,84.912307,-33.991074,357.466751,None,29.922213,unknown,2.459523e+06,59522.214123
25238,2021110300450,AT_O_20211103_000450,20211103,450,2021-11-04 05:08:54.415326,2021-11-04 05:08:56.651,science,HD 37795,FELH0600~holo4_003,29.759992,2.0,84.858917,-33.996452,357.467396,None,29.759992,unknown,2.459523e+06,59522.214519
25239,2021110300451,AT_O_20211103_000451,20211103,451,2021-11-04 05:11:43.526616,2021-11-04 05:11:44.751,science,HD 37795,FELH0600~holo4_003,29.208396,1.0,84.912423,-34.074294,357.966363,None,29.208396,unknown,2.459523e+06,59522.216476
25240,2021110300452,AT_O_20211103_000452,20211103,452,2021-11-04 05:12:19.215475,2021-11-04 05:12:20.455,science,HD 37795,FELH0600~holo4_003,29.095972,1.0,84.925386,-34.039079,357.965341,None,29.095972,unknown,2.459523e+06,59522.216889
25241,2021110300453,AT_O_20211103_000453,20211103,453,2021-11-04 05:12:52.522226,2021-11-04 05:12:53.757,science,HD 37795,FELH0600~holo4_003,28.980319,1.0,84.923574,-34.038003,357.966044,None,28.980319,unknown,2.459523e+06,59522.217275
25243,2021110300455,AT_O_20211103_000455,20211103,455,2021-11-04 05:17:12.322792,2021-11-04 05:17:13.556,science,HD 38666,FELH0600~holo4_003,29.505123,1.0,86.499726,-32.306347,354.723354,None,29.505123,unknown,2.459523e+06,59522.220282
25244,2021110300456,AT_O_20211103_000456,20211103,456,2021-11-04 05:17:52.968290,2021-11-04 05:17:54.212,science,HD 38666,FELH0600~holo4_003,29.375247,1.0,86.508605,-32.271477,354.725128,None,29.375247,unknown,2.459523e+06,59522.220752
25250,2021110300462,AT_O_20211103_000462,20211103,462,2021-11-04 05:25:31.608019,2021-11-04 05:26:01.840,science,HD 38666,FELH0600~holo4_003,27.759744,30.0,86.508595,-32.270625,354.723764,None,27.759744,unknown,2.459523e+06,59522.226060
25257,2021110300468,AT_O_20211103_000468,20211103,468,2021-11-04 05:29:35.261194,2021-11-04 05:30:05.497,science,HD 38666,FELH0600~holo4_003,26.900151,30.0,86.508557,-32.270622,354.723659,None,26.900151,unknown,2.459523e+06,59522.228880


In [35]:
if N>=2:
    n1=len(all_def_science_selected_final[1])
    display(all_def_science_selected_final[1])

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
25247,2021110300459,AT_O_20211103_000459,20211103,459,2021-11-04 05:23:15.099847,2021-11-04 05:23:45.334,science,HD 38666,empty~holo4_003,28.24208,30.0,86.508515,-32.270641,354.723745,None,28.24208,unknown,2.459523e+06,59522.224480
25254,2021110300465,AT_O_20211103_000465,20211103,465,2021-11-04 05:27:18.411466,2021-11-04 05:27:48.646,science,HD 38666,empty~holo4_003,27.382811,30.0,86.50853,-32.270625,354.723849,None,27.382811,unknown,2.459523e+06,59522.227296
25260,2021110300471,AT_O_20211103_000471,20211103,471,2021-11-04 05:31:21.800145,2021-11-04 05:31:52.035,science,HD 38666,empty~holo4_003,26.526583,30.0,86.508552,-32.270615,354.723461,None,26.526583,unknown,2.459523e+06,59522.230113
25266,2021110300477,AT_O_20211103_000477,20211103,477,2021-11-04 05:35:29.983963,2021-11-04 05:36:00.220,science,HD 38666,empty~holo4_003,25.652265,30.0,86.508558,-32.27063,354.724046,None,25.652265,unknown,2.459523e+06,59522.232986
25272,2021110300483,AT_O_20211103_000483,20211103,483,2021-11-04 05:39:32.920468,2021-11-04 05:40:03.168,science,HD 38666,empty~holo4_003,24.795164,30.0,86.508554,-32.270624,354.724115,None,24.795164,unknown,2.459523e+06,59522.235798
25278,2021110300489,AT_O_20211103_000489,20211103,489,2021-11-04 05:43:36.197501,2021-11-04 05:44:06.431,science,HD 38666,empty~holo4_003,23.93414,30.0,86.508511,-32.270625,354.723861,None,23.93414,unknown,2.459523e+06,59522.238613
25284,2021110300495,AT_O_20211103_000495,20211103,495,2021-11-04 05:47:45.319692,2021-11-04 05:48:15.546,science,HD 38666,empty~holo4_003,23.055235,30.0,86.508571,-32.270593,354.722389,None,23.055235,unknown,2.459523e+06,59522.241497
25290,2021110300501,AT_O_20211103_000501,20211103,501,2021-11-04 05:51:48.500295,2021-11-04 05:52:18.731,science,HD 38666,empty~holo4_003,22.19714,30.0,86.508523,-32.270605,354.723577,None,22.19714,unknown,2.459523e+06,59522.244311
25296,2021110300507,AT_O_20211103_000507,20211103,507,2021-11-04 05:55:51.930540,2021-11-04 05:56:22.154,science,HD 38666,empty~holo4_003,21.338854,30.0,86.508493,-32.27062,354.723602,None,21.338854,unknown,2.459523e+06,59522.247129
25302,2021110300513,AT_O_20211103_000513,20211103,513,2021-11-04 06:00:24.075636,2021-11-04 06:00:54.312,science,HD 38666,empty~holo4_003,20.378,30.0,86.508515,-32.270743,354.723787,None,20.378,unknown,2.459523e+06,59522.250279


In [36]:
if N>=3:
    n2=len(all_def_science_selected_final[2])
    display(all_def_science_selected_final[2])

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
25248,2021110300460,AT_O_20211103_000460,20211103,460,2021-11-04 05:24:00.904254,2021-11-04 05:24:31.132,science,HD 38666,BG40~holo4_003,28.080096,30.0,86.508482,-32.270632,354.723756,None,28.080096,unknown,2.459523e+06,59522.225010
25255,2021110300466,AT_O_20211103_000466,20211103,466,2021-11-04 05:28:04.655225,2021-11-04 05:28:34.884,science,HD 38666,BG40~holo4_003,27.220839,30.0,86.508624,-32.270623,354.72379,None,27.220839,unknown,2.459523e+06,59522.227832
25261,2021110300472,AT_O_20211103_000472,20211103,472,2021-11-04 05:32:07.608595,2021-11-04 05:32:37.853,science,HD 38666,BG40~holo4_003,26.364422,30.0,86.508525,-32.270609,354.723692,None,26.364422,unknown,2.459523e+06,59522.230644
25267,2021110300478,AT_O_20211103_000478,20211103,478,2021-11-04 05:36:15.838015,2021-11-04 05:36:46.074,science,HD 38666,BG40~holo4_003,25.490008,30.0,86.50851,-32.270621,354.723686,None,25.490008,unknown,2.459523e+06,59522.233517
25273,2021110300484,AT_O_20211103_000484,20211103,484,2021-11-04 05:40:18.833679,2021-11-04 05:40:49.071,science,HD 38666,BG40~holo4_003,24.632855,30.0,86.508533,-32.270619,354.723792,None,24.632855,unknown,2.459523e+06,59522.236329
25279,2021110300490,AT_O_20211103_000490,20211103,490,2021-11-04 05:44:22.424326,2021-11-04 05:44:52.658,science,HD 38666,BG40~holo4_003,23.771771,30.0,86.508496,-32.270626,354.723813,None,23.771771,unknown,2.459523e+06,59522.239148
25285,2021110300496,AT_O_20211103_000496,20211103,496,2021-11-04 05:48:31.222665,2021-11-04 05:49:01.454,science,HD 38666,BG40~holo4_003,22.892863,30.0,86.508616,-32.270615,354.723616,None,22.892863,unknown,2.459523e+06,59522.242028
25291,2021110300502,AT_O_20211103_000502,20211103,502,2021-11-04 05:52:34.718869,2021-11-04 05:53:04.957,science,HD 38666,BG40~holo4_003,22.034692,30.0,86.50853,-32.27062,354.723527,None,22.034692,unknown,2.459523e+06,59522.244846
25297,2021110300508,AT_O_20211103_000508,20211103,508,2021-11-04 05:56:37.807722,2021-11-04 05:57:08.038,science,HD 38666,BG40~holo4_003,21.17642,30.0,86.508559,-32.270625,354.723398,None,21.17642,unknown,2.459523e+06,59522.247660
25303,2021110300514,AT_O_20211103_000514,20211103,514,2021-11-04 06:01:10.281467,2021-11-04 06:01:40.510,science,HD 38666,BG40~holo4_003,20.211981,30.0,86.508544,-32.270611,354.723377,None,20.211981,unknown,2.459523e+06,59522.250813


In [37]:
if N>=4:
    n3=len(all_def_science_selected_final[3])
    display(all_def_science_selected_final[3])

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
25249,2021110300461,AT_O_20211103_000461,20211103,461,2021-11-04 05:24:45.687413,2021-11-04 05:25:15.911,science,HD 38666,SDSSg~holo4_003,27.921729,30.0,86.508584,-32.270606,354.723684,None,27.921729,unknown,2.459523e+06,59522.225529
25256,2021110300467,AT_O_20211103_000467,20211103,467,2021-11-04 05:28:49.384842,2021-11-04 05:29:19.621,science,HD 38666,SDSSg~holo4_003,27.062237,30.0,86.508558,-32.270624,354.723616,None,27.062237,unknown,2.459523e+06,59522.228349
25262,2021110300473,AT_O_20211103_000473,20211103,473,2021-11-04 05:32:52.464218,2021-11-04 05:33:22.702,science,HD 38666,SDSSg~holo4_003,26.205784,30.0,86.508515,-32.270622,354.723793,None,26.205784,unknown,2.459523e+06,59522.231163
25268,2021110300479,AT_O_20211103_000479,20211103,479,2021-11-04 05:37:00.656020,2021-11-04 05:37:30.884,science,HD 38666,SDSSg~holo4_003,25.331445,30.0,86.508679,-32.270614,354.723854,None,25.331445,unknown,2.459523e+06,59522.234035
25274,2021110300485,AT_O_20211103_000485,20211103,485,2021-11-04 05:41:03.674250,2021-11-04 05:41:33.907,science,HD 38666,SDSSg~holo4_003,24.474113,30.0,86.508572,-32.270626,354.723817,None,24.474113,unknown,2.459523e+06,59522.236848
25280,2021110300491,AT_O_20211103_000491,20211103,491,2021-11-04 05:45:07.199199,2021-11-04 05:45:37.439,science,HD 38666,SDSSg~holo4_003,23.612994,30.0,86.508566,-32.27062,354.723774,None,23.612994,unknown,2.459523e+06,59522.239667
25286,2021110300497,AT_O_20211103_000497,20211103,497,2021-11-04 05:49:16.082815,2021-11-04 05:49:46.317,science,HD 38666,SDSSg~holo4_003,22.737461,30.0,86.508562,-32.270616,354.723577,None,22.737461,unknown,2.459523e+06,59522.242547
25292,2021110300503,AT_O_20211103_000503,20211103,503,2021-11-04 05:53:19.502252,2021-11-04 05:53:49.726,science,HD 38666,SDSSg~holo4_003,21.875809,30.0,86.508585,-32.270614,354.723782,None,21.875809,unknown,2.459523e+06,59522.245365
25298,2021110300509,AT_O_20211103_000509,20211103,509,2021-11-04 05:57:22.673073,2021-11-04 05:57:52.909,science,HD 38666,SDSSg~holo4_003,21.017505,30.0,86.508613,-32.270612,354.723513,None,21.017505,unknown,2.459523e+06,59522.248179
25304,2021110300515,AT_O_20211103_000515,20211103,515,2021-11-04 06:01:55.009572,2021-11-04 06:02:25.242,science,HD 38666,SDSSg~holo4_003,20.056521,30.0,86.508532,-32.270634,354.723831,None,20.056521,unknown,2.459523e+06,59522.251331


In [38]:
if N>=5:
    n4=len(all_def_science_selected_final[4])
    display(all_def_science_selected_final[4])

In [39]:
df_forsave = pd.concat(all_def_science_selected_final)

In [40]:
filename_out_csv = "holosummary_"+ str(date_sel) + ".csv"
filename_out_excel = "holosummary_"+ str(date_sel) + ".xlsx"

In [41]:
filename_out_csv

'holosummary_20211103.csv'

In [42]:
df_forsave.to_csv(filename_out_csv) 

In [43]:
#! pip install openpyxl

In [44]:
df_forsave.to_excel(filename_out_excel) 